Khai báo thư viện cần dùng trong bài toán

In [1]:
from pyvi import ViTokenizer
from tqdm import tqdm
import numpy as np
import gensim 
from sklearn import preprocessing
import os 
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import FastText
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

Tải lên dữ liệu từ thư mục chứa

In [2]:
dir_path = os.path.dirname(os.path.realpath(os.getcwd())) #lấy thư mục làm việc hiện tại và đường dẫn của nó
dir_path = os.path.join(dir_path, 'Data')

def get_data(folder_path):      #Định nghĩa hàm get_data(đường dẫn chứa data)
    X = []
    y = []
    dirs = os.listdir(folder_path)
    for path in tqdm(dirs):         #hiện tiến trình
        file_paths = os.listdir(os.path.join(folder_path, path))    #lấy danh sách các tệp trong 'folder_path'
        for file_path in tqdm(file_paths):
            with open(os.path.join(folder_path, path, file_path), 'r', encoding="utf-16") as f:
                lines = f.readlines()
                lines = ' '.join(lines)
                lines = gensim.utils.simple_preprocess(lines)   #xử lý các ký tự đặc biệt
                lines = ' '.join(lines)
                lines = ViTokenizer.tokenize(lines)     #Tách từ

                X.append(lines)     #thêm vban đã tách vào X
                y.append(path)      #Thêm nhãn vào Y

    return X, y

train_path = os.path.join(dir_path, r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\Train\new train')
X_data, y_data = get_data(train_path)


100%|██████████| 4/4 [00:14<00:00,  3.71s/it]


In [3]:
pickle.dump(X_data, open(r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\X_data.pkl', 'wb'))        #Binary
pickle.dump(y_data, open(r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\y_data.pkl', 'wb'))


In [4]:
test_path = os.path.join(dir_path, r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\Test\new test')
X_test, y_test = get_data(test_path)

pickle.dump(X_test, open(r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\X_test.pkl', 'wb'))
pickle.dump(y_test, open(r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\y_test.pkl', 'wb'))


100%|██████████| 4/4 [00:11<00:00,  2.90s/it]


In [5]:
X_data = pickle.load(open(r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\X_data.pkl', 'rb'))
y_data = pickle.load(open(r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\y_data.pkl', 'rb'))

X_test = pickle.load(open(r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\X_test.pkl', 'rb'))
y_test = pickle.load(open(r'E:\hoc hanh\THUY LOI HOC\NLP\btl1\y_test.pkl', 'rb'))


In [27]:
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=1000)
tfidf_vect.fit(X_data) #Học từ vựng
X_data_tfidf =  tfidf_vect.transform(X_data)        #biến đổi dữ liệu đào tạo thành ma trận TF-IDF
# assume that we don't have test set before
X_test_tfidf =  tfidf_vect.transform(X_test)


In [28]:
from gensim.models import FastText
import os

# Đường dẫn đến file .bin
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
fasttext_model_path = os.path.join(dir_path, r"E:\hoc hanh\THUY LOI HOC\NLP\btl1\cc.vi.300.bin\cc.vi.300.bin")

# Tải mô hình FastText
model = FastText.load_fasttext_format(fasttext_model_path)

def get_word2vec_data(X):
    word2vec_data = []
    for x in X:
        sentence = []
        for word in x.split(" "):
            if word in model.wv.key_to_index:
                sentence.append(model.wv[word])
        word2vec_data.append(sentence)
    return word2vec_data

# Giả sử X_data và X_test là dữ liệu của bạn
X_data_w2v = get_word2vec_data(X_data)
X_test_w2v = get_word2vec_data(X_test)


C:\Users\COMPUTER\AppData\Local\Temp\ipykernel_22832\3068195609.py:9: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  model = FastText.load_fasttext_format(fasttext_model_path)


In [29]:
encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_data)
y_test_n = encoder.fit_transform(y_test)

encoder.classes_ 


array(['Gioi tinh', 'Hackers va Virus', 'The gioi tre', 'Thoi trang'],
      dtype='<U16')

Xây dựng model LSTM

In [30]:
from sklearn.metrics import accuracy_score
from keras.layers import Dropout
# Chuyển đổi dữ liệu đầu vào thành dạng ma trận
max_len = 100 # Độ dài tối đa của mỗi câu
X_data_pad = pad_sequences(X_data_w2v, maxlen=max_len, dtype='float32')
X_test_pad = pad_sequences(X_test_w2v, maxlen=max_len, dtype='float32')

# Xây dựng mô hình LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(max_len, 300))) # 300 là số chiều của embedding vector
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(len(encoder.classes_), activation='softmax')) # Số lớp đầu ra

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Huấn luyện mô hình
history = model.fit(X_data_pad, y_data_n, batch_size=32, epochs=10, validation_data=(X_test_pad, y_test_n))

# Dự đoán xác suất cho mỗi lớp trên tập test
y_pred_prob = model.predict(X_test_pad)

# Lấy ra lớp có xác suất cao nhất cho mỗi mẫu
y_pred = np.argmax(y_pred_prob, axis=1)

# In ra train accuracy
train_accuracy = history.history['accuracy'][-1]
print("Train accuracy:", train_accuracy)

# In ra validation accuracy
val_accuracy = history.history['val_accuracy'][-1]
print("Validation accuracy:", val_accuracy)

# Tính và in ra test accuracy
test_accuracy = accuracy_score(y_test_n, y_pred)
print("Test accuracy:", test_accuracy)


Epoch 1/10
46/46 [==============================] - 15s 167ms/step - loss: 1.1497 - accuracy: 0.4622 - val_loss: 0.8930 - val_accuracy: 0.5666
Epoch 2/10
46/46 [==============================] - 7s 150ms/step - loss: 0.7363 - accuracy: 0.6967 - val_loss: 0.8009 - val_accuracy: 0.7549
Epoch 3/10
46/46 [==============================] - 6s 132ms/step - loss: 0.6226 - accuracy: 0.7252 - val_loss: 0.8280 - val_accuracy: 0.6139
Epoch 4/10
46/46 [==============================] - 5s 118ms/step - loss: 0.8974 - accuracy: 0.5808 - val_loss: 0.9756 - val_accuracy: 0.5162
Epoch 5/10
46/46 [==============================] - 6s 136ms/step - loss: 0.7085 - accuracy: 0.6836 - val_loss: 0.6546 - val_accuracy: 0.7250
Epoch 6/10
46/46 [==============================] - 6s 123ms/step - loss: 0.4291 - accuracy: 0.8397 - val_loss: 0.3848 - val_accuracy: 0.8589
Epoch 7/10
46/46 [==============================] - 6s 140ms/step - loss: 0.3011 - accuracy: 0.8917 - val_loss: 0.3126 - val_accuracy: 0.8905
Epoch